# AI-enhanced Speech Analytics Process

## Implemeting speech recognition

In [2]:
import speech_recognition as sr
from pydub import AudioSegment
from pydub.playback import play

In [4]:
recognizer = sr.Recognizer()

try:
  audio_full_filename = 'test.flac'
  audio_filename = audio_full_filename[0:audio_full_filename.rfind('.')]
  raw_audios_directory = 'audios/raw'
  converted_audios_directory = 'audios/converted'

  audio_file = AudioSegment.from_file(f'{raw_audios_directory}/{audio_full_filename}')
  test = audio_file.export(f'{converted_audios_directory}/{audio_filename}.wav', format='wav')

  with sr.AudioFile(f'{converted_audios_directory}/{audio_filename}.wav') as source:
    audio_data = recognizer.record(source)
    text = recognizer.recognize_google(audio_data)
    print(f'Recognized text: {text}')

except sr.UnknownValueError():
  recognizer = sr.Recognizer()


Recognized text: test test test test test input devices configuration output devices recording
